In [36]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [37]:
ex1 = pd.read_parquet('train/1.회원정보/2018_회원정보_전체.parquet')

In [38]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '남녀구분코드', '연령', 'Segment', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '거주시도명', '직장시도명', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용카드수_체크_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '이용금액_R3M_체크_가족', '_1순위카드이용금액', '_1순위카드이용건수', '_1순위신용체크구분', '_2순위카드이용금액', '_2순위카드이용건수', '_2순위신용체크구분', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비발생카드수_B0M', '연회비할인카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '청구금액_기본연회

In [39]:
cols = ['가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분', '연회비발생카드수_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', 'Life_Stage']  # 실제 열 이름으로 바꿔주세요

In [40]:
mapping = {
    'L사': 0,
    'K사': 1,
    'S사': 2
}
ex1['가입통신회사코드'] = (
    ex1['가입통신회사코드']
      .map(mapping)        # L사→0, K사→1, S사→2, 그 외→NaN
      .fillna(-1)          # NaN→-1
      .astype(int)         # 정수형
)

In [41]:
ex1['연령'] = ex1['연령'].astype(str).str.replace('대', '', regex=False)

# 2) 숫자로 변환, 에러는 NaN→fillna(-1)→정수형
ex1['연령'] = (
    pd.to_numeric(ex1['연령'], errors='coerce')
      .fillna(-1)
      .astype(int)
)


In [42]:

regions = [
    '서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

region_type = CategoricalDtype(categories=regions, ordered=True)

for col in ['거주시도명', '직장시도명']:
    ex1[col] = (
        ex1[col]
           .astype(str)                   # 혹시 숫자 타입 섞여 있어도 문자열로
           .astype(region_type)           # 카테고리로 변환
           .cat.codes                     # 0~16, 리스트 외 값은 -1
    )

In [43]:
count_cols = [
    '연회비발생카드수_B0M',
    '상품관련면제카드수_B0M',
    '임직원면제카드수_B0M',
    '우수회원면제카드수_B0M',
    '기타면제카드수_B0M'
]

for col in count_cols:
    ex1[col] = (
        ex1[col]
          .astype(str)  # None → 'None', NaN → 'nan'
          .str.replace('개', '', regex=False)  # '0개' → '0', 'None' → 'None'
    )
    # 숫자로 변환, 에러는 NaN 처리 → NaN → -1 로 채우기 → 정수형
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [44]:
le_cols = ['_1순위신용체크구분', '_2순위신용체크구분', 'Life_Stage']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes 

In [45]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '남녀구분코드', '연령', 'Segment', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '거주시도명', '직장시도명', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용카드수_체크_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '이용금액_R3M_체크_가족', '_1순위카드이용금액', '_1순위카드이용건수', '_1순위신용체크구분', '_2순위카드이용금액', '_2순위카드이용건수', '_2순위신용체크구분', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비발생카드수_B0M', '연회비할인카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '청구금액_기본연회

In [67]:
ex1 = ex1.fillna(-1)
ex1

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능_카드론,입회일자_신용,수신거부여부_TM,수신거부여부_SMS,가입통신회사코드,탈회횟수_누적,직장시도명,이용카드수_신용체크,이용금액_R3M_신용체크,최종유효년월_신용_이용가능,최종카드발급일자,이용여부_3M_해외겸용_본인,Life_Stage,최종카드발급경과월,Segment_encoded
0,201807,TRAIN_000000,2,40,D,0,20130101,0,0,0,1,0,1,196,202110.0,20160912.0,0,5,22,3
1,201807,TRAIN_000001,1,30,E,1,20170801,0,0,1,1,8,1,13475,202112.0,20170122.0,0,4,18,4
2,201807,TRAIN_000002,1,30,C,0,20080401,0,0,2,1,0,1,23988,202111.0,20161113.0,0,6,20,2
3,201807,TRAIN_000003,2,40,D,0,20160501,0,0,0,1,1,1,3904,202201.0,20170205.0,1,5,17,3
4,201807,TRAIN_000004,2,40,E,1,20180601,0,0,1,1,4,1,1190,202201.0,20170409.0,1,4,15,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,2,-1,E,1,20010701,0,0,2,0,6,1,10755,202110.0,20150902.0,1,1,39,4
2399996,201812,TRAIN_399996,2,50,D,1,20170701,0,0,0,1,3,1,27636,202110.0,20161201.0,1,5,24,3
2399997,201812,TRAIN_399997,1,30,C,0,20090501,1,1,2,0,0,1,23187,202110.0,20170609.0,1,6,18,2
2399998,201812,TRAIN_399998,1,40,E,1,20130101,1,1,2,0,1,0,0,202202.0,20160928.0,0,4,27,4


In [47]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.01 ].index.tolist()
low_var

['유효카드수_체크_가족',
 '이용가능카드수_체크_가족',
 '이용카드수_체크_가족',
 '이용금액_R3M_체크_가족',
 '연회비발생카드수_B0M',
 '연회비할인카드수_B0M',
 '할인금액_기본연회비_B0M',
 '할인금액_제휴연회비_B0M',
 '상품관련면제카드수_B0M',
 '임직원면제카드수_B0M',
 '우수회원면제카드수_B0M',
 '기타면제카드수_B0M']

In [48]:
cols_to_drop = ['회원여부_이용가능',
 '소지여부_신용',
 '유효카드수_체크_가족',
 '이용가능카드수_체크_가족',
 '이용카드수_체크_가족',
 '이용금액_R3M_체크_가족',
 '_1순위신용체크구분',
 '연회비발생카드수_B0M',
 '연회비할인카드수_B0M',
 '할인금액_기본연회비_B0M',
 '할인금액_제휴연회비_B0M',
 '상품관련면제카드수_B0M',
 '임직원면제카드수_B0M',
 '우수회원면제카드수_B0M',
 '기타면제카드수_B0M']
ex1.drop(columns=cols_to_drop, inplace=True)

In [51]:
ex1

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능_카드론,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,입회경과개월수_신용,...,최종카드발급일자,보유여부_해외겸용_본인,이용가능여부_해외겸용_본인,이용여부_3M_해외겸용_본인,보유여부_해외겸용_신용_본인,이용가능여부_해외겸용_신용_본인,이용여부_3M_해외겸용_신용_본인,Life_Stage,최종카드발급경과월,Segment_encoded
0,201807,TRAIN_000000,2,40,D,0,1,1,20130101,67,...,20160912.0,0,0,0,0,0,0,5,22,3
1,201807,TRAIN_000001,1,30,E,1,1,1,20170801,12,...,20170122.0,0,0,0,0,0,0,4,18,4
2,201807,TRAIN_000002,1,30,C,0,1,1,20080401,124,...,20161113.0,1,1,0,0,0,0,6,20,2
3,201807,TRAIN_000003,2,40,D,0,2,2,20160501,27,...,20170205.0,1,1,1,1,1,1,5,17,3
4,201807,TRAIN_000004,2,40,E,1,1,1,20180601,2,...,20170409.0,1,1,1,0,0,0,4,15,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,2,-1,E,1,1,1,20010701,209,...,20150902.0,1,1,1,1,1,0,1,39,4
2399996,201812,TRAIN_399996,2,50,D,1,1,1,20170701,17,...,20161201.0,1,1,1,1,1,1,5,24,3
2399997,201812,TRAIN_399997,1,30,C,0,1,1,20090501,115,...,20170609.0,1,1,1,1,1,1,6,18,2
2399998,201812,TRAIN_399998,1,40,E,1,1,1,20130101,71,...,20160928.0,0,0,0,0,0,0,4,27,4


In [49]:
missing_mask = ex1.isna() | (ex1 == -1)
missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.2].index.tolist()

high_const_cols = []
threshold_const = 0.8

for col in ex1.columns:
    top_ratio = ex1[col].value_counts(normalize=True, dropna=False).values[0]
    if top_ratio > threshold_const:
        high_const_cols.append(col)

to_drop = list(set(high_na + high_const_cols))

if 'Segment' in to_drop:
    to_drop.remove('Segment')

print("삭제 대상 컬럼 (결측>20% 또는 동일값>80%):", to_drop)

ex1.drop(columns=to_drop, inplace=True)


삭제 대상 컬럼 (결측>20% 또는 동일값>80%): ['마케팅동의여부', '이용가능카드수_신용_가족', '이용거절여부_카드론', '유효카드수_신용_가족', '_2순위신용체크구분', '이용카드수_신용_가족', '제휴연회비_B0M', '청구금액_기본연회비_B0M', '카드신청건수', '회원여부_이용가능_CA', '회원여부_연체', '이용카드수_체크', '최종유효년월_신용_이용', '이용금액_R3M_체크', '탈회횟수_발급6개월이내', '이용금액_R3M_신용_가족', '동의여부_한도증액안내', '기본연회비_B0M', '청구금액_제휴연회비_B0M', '탈회횟수_발급1년이내']


In [58]:
import pandas as pd
import numpy as np

num_df = ex1.select_dtypes(include=[np.number]).dropna()

corr = num_df.corr().abs()

high_corr_pairs = (
    corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        .stack()
        .reset_index()
)
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > 0.7]

if not np.issubdtype(ex1['Segment'].dtype, np.number):
    segment_map = {label: idx for idx, label in enumerate(sorted(ex1['Segment'].unique()))}
    ex1['Segment_encoded'] = ex1['Segment'].map(segment_map)
else:
    ex1['Segment_encoded'] = ex1['Segment']

segment_corr = ex1[num_df.columns].corrwith(ex1['Segment_encoded']).abs()

high_corr_pairs['Corr_with_Segment_1'] = high_corr_pairs['Feature_1'].map(segment_corr)
high_corr_pairs['Corr_with_Segment_2'] = high_corr_pairs['Feature_2'].map(segment_corr)

high_corr_pairs = high_corr_pairs.sort_values(by='Correlation', ascending=False).reset_index(drop=True)

print(f"▶ 상관계수 0.7 초과 변수쌍 수: {len(high_corr_pairs)}")
display(high_corr_pairs)


▶ 상관계수 0.7 초과 변수쌍 수: 47


,Feature_1,Feature_2,Correlation,Corr_with_Segment_1,Corr_with_Segment_2
0,입회일자_신용,입회경과개월수_신용,0.999016,0.252289,0.252240
1,유효카드수_체크,이용가능카드수_체크,0.994470,0.147368,0.146998
2,수신거부여부_DM,수신거부여부_메일,0.960796,0.104624,0.106260
3,유효카드수_신용체크,이용가능카드수_신용체크,0.950798,0.339723,0.352211
4,_2순위카드이용금액,_2순위카드이용건수,0.949612,0.400225,0.376494
5,이용금액_R3M_신용체크,이용금액_R3M_신용,0.946124,0.622827,0.589032
6,이용금액_R3M_신용,_1순위카드이용금액,0.934391,0.589032,0.573870
7,이용카드수_신용체크,이용카드수_신용,0.927533,0.397713,0.376918
8,보유여부_해외겸용_본인,이용가능여부_해외겸용_본인,0.923950,0.143593,0.159198
9,유효카드수_신용,이용가능카드수_신용,0.910991,0.342925,0.353014


In [59]:
to_drop = []

for _, row in high_corr_pairs.iterrows():
    f1, f2 = row['Feature_1'], row['Feature_2']
    c1, c2 = row['Corr_with_Segment_1'], row['Corr_with_Segment_2']
    
    if pd.isna(c1) or pd.isna(c2):
        continue
    
    if c1 < c2:
        to_drop.append(f1)
    else:
        to_drop.append(f2)

to_drop = list(set(to_drop))

# 결과 출력
print(f"▶ 제거 대상 피처 수: {len(to_drop)}")
print("제거할 피처 목록:")
print(to_drop)

▶ 제거 대상 피처 수: 24
제거할 피처 목록:
['_2순위카드이용건수', '이용가능여부_해외겸용_본인', '수신거부여부_메일', '유효카드수_체크', '_2순위카드이용금액', '_1순위카드이용건수', '유효카드수_신용', '이용가능카드수_신용체크', '보유여부_해외겸용_신용_본인', '최종탈회후경과월', '소지카드수_이용가능_신용', '이용여부_3M_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '_1순위카드이용금액', '이용금액_R3M_신용', '이용가능카드수_체크', '유효카드수_신용체크', '거주시도명', '입회경과개월수_신용', '이용카드수_신용', '소지카드수_유효_신용', '이용가능카드수_신용', '보유여부_해외겸용_본인', '수신거부여부_DM']


In [60]:
cols_to_drop = ['_2순위카드이용건수', '이용가능여부_해외겸용_본인', '수신거부여부_메일', '유효카드수_체크', '_2순위카드이용금액', '_1순위카드이용건수', '유효카드수_신용', '이용가능카드수_신용체크', '보유여부_해외겸용_신용_본인', '최종탈회후경과월', '소지카드수_이용가능_신용', '이용여부_3M_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '_1순위카드이용금액', '이용금액_R3M_신용', '이용가능카드수_체크', '유효카드수_신용체크', '거주시도명', '입회경과개월수_신용', '이용카드수_신용', '소지카드수_유효_신용', '이용가능카드수_신용', '보유여부_해외겸용_본인', '수신거부여부_DM']
ex1.drop(columns=cols_to_drop, inplace=True)

In [62]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF > 10인 컬럼들: []
Empty DataFrame
Columns: [feature, VIF]
Index: []


In [63]:
ex1

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능_카드론,입회일자_신용,수신거부여부_TM,수신거부여부_SMS,가입통신회사코드,탈회횟수_누적,직장시도명,이용카드수_신용체크,이용금액_R3M_신용체크,최종유효년월_신용_이용가능,최종카드발급일자,이용여부_3M_해외겸용_본인,Life_Stage,최종카드발급경과월,Segment_encoded
0,201807,TRAIN_000000,2,40,D,0,20130101,0,0,0,1,0,1,196,202110.0,20160912.0,0,5,22,3
1,201807,TRAIN_000001,1,30,E,1,20170801,0,0,1,1,8,1,13475,202112.0,20170122.0,0,4,18,4
2,201807,TRAIN_000002,1,30,C,0,20080401,0,0,2,1,0,1,23988,202111.0,20161113.0,0,6,20,2
3,201807,TRAIN_000003,2,40,D,0,20160501,0,0,0,1,1,1,3904,202201.0,20170205.0,1,5,17,3
4,201807,TRAIN_000004,2,40,E,1,20180601,0,0,1,1,4,1,1190,202201.0,20170409.0,1,4,15,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,2,-1,E,1,20010701,0,0,2,0,6,1,10755,202110.0,20150902.0,1,1,39,4
2399996,201812,TRAIN_399996,2,50,D,1,20170701,0,0,0,1,3,1,27636,202110.0,20161201.0,1,5,24,3
2399997,201812,TRAIN_399997,1,30,C,0,20090501,1,1,2,0,0,1,23187,202110.0,20170609.0,1,6,18,2
2399998,201812,TRAIN_399998,1,40,E,1,20130101,1,1,2,0,1,0,0,202202.0,20160928.0,0,4,27,4


In [64]:
from sklearn.preprocessing import LabelEncoder

df_corr = ex1.drop(columns=['기준년월', 'ID']).copy()

le = LabelEncoder()
df_corr['Segment_encoded'] = le.fit_transform(df_corr['Segment'])

correlations = df_corr.corr(numeric_only=True)['Segment_encoded'].drop('Segment_encoded')

selected_cols = correlations[correlations.abs() > 0.1].index.tolist()

final_cols = ['기준년월', 'ID','Segment'] + selected_cols

ex1_filtered = ex1[final_cols]


In [65]:
cols = ex1_filtered.columns.tolist()
cols

['기준년월',
 'ID',
 'Segment',
 '입회일자_신용',
 '수신거부여부_TM',
 '이용카드수_신용체크',
 '이용금액_R3M_신용체크',
 '최종유효년월_신용_이용가능',
 '이용여부_3M_해외겸용_본인']

In [66]:
ex1_filtered.to_parquet('회원_전처리_Segment.parquet', index=False)